In [3]:
%load_ext autoreload
%autoreload 1

In [4]:
import pandas as pd
import numpy as np

import sys 
sys.path.append("../") 

%aimport p2v
%aimport preprocessing
%aimport utils

from utils import prinT
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.options.display.max_columns = None
pd.set_option('max_colwidth', None)

# Make

In [6]:
preprocessing.make_MAG_venue_info_df()

2023-06-02 15:37:28 -- start loading paper_detailed.pkl
2023-06-02 15:40:19 -- finish
INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
2023-06-02 15:40:19 -- preparing VID for papers...


/home/zqlyu/p2v/preprocessing.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MAG_venue_info_df.loc[:,'VenueID'] = MAG_paper_detailed_df[['DocType','JournalID','ConferenceSeriesID']].parallel_apply(lambda x:
/home/zqlyu/p2v/preprocessing.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MAG_venue_info_df['VenueID'] = pd.to_numeric(df['VenueID'], errors='coerce')
/home/zqlyu/p2v/preprocessing.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the

2023-06-02 15:45:15 -- finish. MAG_venue_info_df saved


,VenueType,OriginalVenue
VenueID,,
3006142753,Journal,Proceedings of The Royal Society of London
194691280,Journal,Annalen der Physik
177993953,Journal,Journal of Literacy Research
168309380,Journal,Philosophical Magazine
93133213,Journal,Western Journal of Communication
...,...,...
938070683,Journal,Corporate Philanthropy Report
946020915,Journal,Africa Research Bulletin
6224198,Journal,Loisir & Societe-society and Leisure


In [11]:
preprocessing.make_labeled_journal_info_df()

2023-06-02 15:58:49 -- finish. labeled_journal_info_df saved


,VenueType,OriginalVenue,ScopusName,ScopusCategory,InceptionYear
VenueID,,,,,
194691280,Journal,Annalen der Physik,Annalen der Physik,General Physics and Astronomy,1790
177993953,Journal,Journal of Literacy Research,Journal of Literacy Research,Linguistics and Language;;Language and Linguistics;;Education,1969
168309380,Journal,Philosophical Magazine,Philosophical Magazine,Condensed Matter Physics,1798
93133213,Journal,Western Journal of Communication,Western Journal of Communication,Language and Linguistics;;Communication,None
179784352,Journal,Journal of Women & Aging,Journal of Women and Aging,Geriatrics and Gerontology;;Gender Studies,None
...,...,...,...,...,...
3207474444,Journal,Journal of Statistics and Data Science Education,Journal of Statistics and Data Science Education,Management Science and Operations Research;;Education;;Statistics and Probability,None
3165397051,Journal,Fixed Point Theory and Algorithms for Sciences and Engineering,Fixed Point Theory and Algorithms for Sciences and Engineering,Applied Mathematics;;Geometry and Topology,None
3205933193,Journal,LHB,LHB: Hydroscience Journal,Water Science and Technology,None


In [9]:
model = p2v.P2V()

2024-11-04 09:54:13 -- start loading Mag_venue_info_df
2024-11-04 09:54:13 -- finish.
2024-11-04 09:54:13 -- start loading labeled_journal_info_df
2024-11-04 09:54:13 -- finish.


In [3]:
model.load_paper_df(full_load=True)

2024-11-01 12:58:57 -- start loading Mag_venue_info_df
2024-11-01 12:58:57 -- finish.
2024-11-01 12:58:57 -- start loading labeled_journal_info_df
2024-11-01 12:58:57 -- finish.
2024-11-01 12:58:57 -- start loading 'paper_df'...
2024-11-01 12:58:57 -- finish.


In [4]:
venue_publication_counts_df = model.paper_df.VenueID.value_counts().rename_axis('VenueID').reset_index(name='PublicationVolume')
model.MAG_venue_info_df = model.MAG_venue_info_df.merge(venue_publication_counts_df, on='VenueID', how='left')
model.MAG_venue_info_df['PublicationVolume'] = model.MAG_venue_info_df['PublicationVolume'].fillna(0).astype(int)
model.MAG_venue_info_df.head()


,VenueID,VenueType,OriginalVenue,PublicationVolume
0,3006142753,Journal,Proceedings of The Royal Society of London,6451
1,194691280,Journal,Annalen der Physik,24120
2,177993953,Journal,Journal of Literacy Research,1300
3,168309380,Journal,Philosophical Magazine,16426
4,93133213,Journal,Western Journal of Communication,761


In [11]:
import os
model.MAG_venue_info_df.to_pickle(os.path.join(model.data_dir, 'MAG_venue_info_df.pkl'))

In [10]:
start_year_list = [1950, 1960, 1970, 1980, 1990, 2000, 2010]
end_year_list = [1959, 1969, 1979, 1989, 1999, 2009, 2021]

for idx, decade in enumerate(['1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s']):
    model.load_paper_df(full_load=False, start_year=start_year_list[idx], end_year=end_year_list[idx])
    venue_publication_counts_df = model.target_paper_df.VenueID.value_counts().rename_axis('VenueID').reset_index(name='PublicationVolume_'+decade)
    model.MAG_venue_info_df = model.MAG_venue_info_df.merge(venue_publication_counts_df, on='VenueID', how='left')
    model.MAG_venue_info_df['PublicationVolume_'+decade] = model.MAG_venue_info_df['PublicationVolume_'+decade].fillna(0).astype(int)
    display(model.MAG_venue_info_df.head())

2024-11-04 09:54:15 -- start loading 'paper_df'...
2024-11-04 09:54:15 -- finish.


,VenueID,VenueType,OriginalVenue,PublicationVolume,PublicationVolume_1950s
0,3006142753,Journal,Proceedings of The Royal Society of London,6451,0
1,194691280,Journal,Annalen der Physik,24120,712
2,177993953,Journal,Journal of Literacy Research,1300,0
3,168309380,Journal,Philosophical Magazine,16426,633
4,93133213,Journal,Western Journal of Communication,761,0


2024-11-04 09:54:15 -- start loading 'paper_df'...
2024-11-04 09:54:15 -- finish.


,VenueID,VenueType,OriginalVenue,PublicationVolume,PublicationVolume_1950s,PublicationVolume_1960s
0,3006142753,Journal,Proceedings of The Royal Society of London,6451,0,0
1,194691280,Journal,Annalen der Physik,24120,712,719
2,177993953,Journal,Journal of Literacy Research,1300,0,23
3,168309380,Journal,Philosophical Magazine,16426,633,2034
4,93133213,Journal,Western Journal of Communication,761,0,0


2024-11-04 09:54:15 -- start loading 'paper_df'...
2024-11-04 09:54:15 -- finish.


...

,VenueID,VenueType,OriginalVenue,PublicationVolume,PublicationVolume_1950s,PublicationVolume_1960s,PublicationVolume_1970s,PublicationVolume_1980s,PublicationVolume_1990s
0,3006142753,Journal,Proceedings of The Royal Society of London,6451,0,0,3,5,20
1,194691280,Journal,Annalen der Physik,24120,712,719,522,590,503
2,177993953,Journal,Journal of Literacy Research,1300,0,23,344,223,235
3,168309380,Journal,Philosophical Magazine,16426,633,2034,2119,1415,1742
4,93133213,Journal,Western Journal of Communication,761,0,0,0,0,157


2024-11-04 09:54:16 -- start loading 'paper_df'...
2024-11-04 09:54:16 -- finish.


,VenueID,VenueType,OriginalVenue,PublicationVolume,PublicationVolume_1950s,PublicationVolume_1960s,PublicationVolume_1970s,PublicationVolume_1980s,PublicationVolume_1990s,PublicationVolume_2000s
0,3006142753,Journal,Proceedings of The Royal Society of London,6451,0,0,3,5,20,31
1,194691280,Journal,Annalen der Physik,24120,712,719,522,590,503,773
2,177993953,Journal,Journal of Literacy Research,1300,0,23,344,223,235,211
3,168309380,Journal,Philosophical Magazine,16426,633,2034,2119,1415,1742,2829
4,93133213,Journal,Western Journal of Communication,761,0,0,0,0,157,195


2024-11-04 09:54:17 -- start loading 'paper_df'...
2024-11-04 09:54:17 -- finish.


,VenueID,VenueType,OriginalVenue,PublicationVolume,PublicationVolume_1950s,PublicationVolume_1960s,PublicationVolume_1970s,PublicationVolume_1980s,PublicationVolume_1990s,PublicationVolume_2000s,PublicationVolume_2010s
0,3006142753,Journal,Proceedings of The Royal Society of London,6451,0,0,3,5,20,31,7
1,194691280,Journal,Annalen der Physik,24120,712,719,522,590,503,773,1514
2,177993953,Journal,Journal of Literacy Research,1300,0,23,344,223,235,211,263
3,168309380,Journal,Philosophical Magazine,16426,633,2034,2119,1415,1742,2829,2829
4,93133213,Journal,Western Journal of Communication,761,0,0,0,0,157,195,408


# Load

In [14]:
model = p2v.P2V()

2023-06-02 15:59:28 -- start loading Mag_venue_info_df
2023-06-02 15:59:28 -- finish.
2023-06-02 15:59:28 -- start loading labeled_journal_info_df
2023-06-02 15:59:28 -- finish.


In [19]:
model.MAG_venue_info_df[model.MAG_venue_info_df.VenueType=='Journal']

,VenueType,OriginalVenue
VenueID,,
3006142753,Journal,Proceedings of The Royal Society of London
194691280,Journal,Annalen der Physik
177993953,Journal,Journal of Literacy Research
168309380,Journal,Philosophical Magazine
93133213,Journal,Western Journal of Communication
...,...,...
3205679767,Journal,RMS: Research in Mathematics & Statistics
938070683,Journal,Corporate Philanthropy Report
946020915,Journal,Africa Research Bulletin


In [20]:
model.MAG_venue_info_df[model.MAG_venue_info_df.VenueType=='Conference']

,VenueType,OriginalVenue
VenueID,,
1130451194,Conference,international conference on communications
2754666132,Conference,reliability physics symposium
2244759375,Conference,international conference on multimedia information networking and security
2238538125,Conference,american control conference
2622911331,Conference,"allerton conference on communication, control, and computing"
...,...,...
2898171188,Conference,international academic multi disciplinary conference
2898554125,Conference,international conference robotics systems and automation engineering
3045323805,Conference,international conference on social computing and social media
